In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

import sys

import pandas as pd
from from_root import from_root
from ib_insync import IB, LimitOrder, util
from loguru import logger

from src.utils import Vars, get_pickle, create_target_opts, place_orders, quick_pf

util.startLoop()

# Set the root
ROOT = from_root()
_vars = Vars(MARKET)
PORT = port = _vars.PORT
DATAPATH = ROOT / 'data' / MARKET
opt_cols = _vars.OPT_COLS[0]

# set vscode path
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>

In [2]:
df = create_target_opts(market=MARKET)

In [3]:
# REMOVE PORTFOLIO SYMBOLS FROM TARGET
# ====================================

# Get the portfolio
with IB().connect(port=PORT) as ib:
    df_pf = quick_pf(ib)

# Remove targets which are already in the portfolio
try:
    if df_pf.empty:
        pass
    else:
        df = df[~df.symbol.isin(set(df_pf.symbol))]
except:
    pass

In [6]:
# REMOVE OPEN ORDER SYMBOLS
# =========================

with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    # ib.sleep(0.3)
    trades = ib.trades()

if trades:
    all_trades_df = (util.df(t.contract for t in trades).join(
        util.df(t.orderStatus
                for t in trades)).join(util.df(t.order for t in trades),
                                        lsuffix="_"))

    all_trades_df.rename({"lastTradeDateOrContractMonth": "expiry"},
                            axis="columns",
                            inplace=True)
    trades_cols = ["conId", "symbol", "secType", "expiry",
        "strike", "right", "orderId", "permId", "action",
        "totalQuantity", "lmtPrice", "status",]

    dfo = all_trades_df[trades_cols]
    df_openords = dfo[all_trades_df.status.isin(_vars. ACTIVE_STATUS)]

else:
    df_openords = pd.DataFrame([{'symbol': None}])
    
if ~df_openords.empty:  # There are some open orders
    df = df[~df.symbol.isin(set(df_openords.symbol))]

In [ ]:
# CLEAN UP FURTHER
# ======================

margins_only = ~df.margin.isnull()
with_expPrice = ~df.expPrice.isnull()

condition = margins_only & with_expPrice

df = df[condition]

In [ ]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]
len(cos)

In [ ]:
df[opt_cols]